<a href="https://colab.research.google.com/github/FREDY129053/Final_theory_work/blob/main/Analitica_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

df = pd.read_csv('/content/answers.csv')
df.head()

,Unnamed: 0,query,занятость,по должности-лемме,по дополнительному признаку,по условиям,общие фразы
0,0,фарпост работа владивосток,NaN,NaN,NaN,NaN,общая фраза
1,1,кофейни вакансии,NaN,NaN,NaN,NaN,общая фраза
2,2,работа разнорабочие часовой,NaN,Рабочий,NaN,NaN,NaN
3,3,личный водитель на день,на неполный день,Водитель,NaN,NaN,NaN
4,4,работа от работодателя персональный водитель,NaN,Водитель,NaN,NaN,NaN


In [2]:
copy_df = df.copy()

copy_df['занятость'] = copy_df['занятость'].apply(lambda x: 1 if type(x) != float else 0)
copy_df['по должности-лемме'] = copy_df['по должности-лемме'].apply(lambda x: 1 if type(x) != float else 0)
copy_df['по дополнительному признаку'] = copy_df['по дополнительному признаку'].apply(lambda x: 1 if type(x) != float else 0)
copy_df['по условиям'] = copy_df['по условиям'].apply(lambda x: 1 if type(x) != float else 0)
copy_df['общие фразы'] = copy_df['общие фразы'].apply(lambda x: 1 if type(x) != float else 0)

copy_df = copy_df[['query', 'занятость', 'по должности-лемме', 'по дополнительному признаку', 'по условиям', 'общие фразы']]
copy_df.head()

,query,занятость,по должности-лемме,по дополнительному признаку,по условиям,общие фразы
0,фарпост работа владивосток,0,0,0,0,1
1,кофейни вакансии,0,0,0,0,1
2,работа разнорабочие часовой,0,1,0,0,0
3,личный водитель на день,1,1,0,0,0
4,работа от работодателя персональный водитель,0,1,0,0,0


In [3]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('russian'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\W', ' ', text)
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(words)

copy_df['query'] = copy_df['query'].apply(preprocess_text)

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    copy_df['query'], copy_df.drop(columns=['query']), test_size=0.2, random_state=42)

# Преобразование текста в векторное представление и обучение модели в одном пайплайне
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000)),
    ('clf', OneVsRestClassifier(LogisticRegression(C=10)))
])

pipeline.fit(X_train, y_train)

# Предсказание классов
y_pred = pipeline.predict(X_test)

# Оценка модели
print(classification_report(y_test, y_pred))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

           0       0.99      0.97      0.98       327
           1       0.99      0.94      0.97      1635
           2       1.00      0.96      0.98       161
           3       1.00      0.99      1.00       139
           4       0.91      0.98      0.95       991

   micro avg       0.97      0.96      0.96      3253
   macro avg       0.98      0.97      0.97      3253
weighted avg       0.97      0.96      0.96      3253
 samples avg       0.96      0.96      0.96      3253



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
!pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 669.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 19.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=d81b7def37466391c996bb686b37df855288a87cd92bf87492f5f8c28fc4824f
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

new_query = "работа экспедитором день"


def lemmatize(text):
    words = text.split() # разбиваем текст на слова
    res = list()
    for word in words:
        p = morph.parse(word)[0]
        res.append(p.normal_form)

    return res

# Функция классификации нового запроса
def classify_query(query):
    query = preprocess_text(query)
    return pipeline.predict([query])


works = [i.lower() for i in['Юрист', 'Аппаратчик', 'Осеменатор']]
query_lemma = lemmatize(new_query)
for i in query_lemma:
  if i in works:
    print(f'По должности-лемме = {i}')

answer = classify_query(new_query)[0]
for i in range(len(answer)):
  if answer[i] == 1:
    print(copy_df.columns.to_list()[1:][i])

занятость


In [4]:
def hyperparameter_tuning(X_train, y_train):
    parameters = {
        'tfidf__max_features': [5000, 10000, 15000],
        'clf__estimator__C': [0.1, 1, 10]
    }

    grid_search = GridSearchCV(pipeline, parameters, cv=3, scoring='f1_macro')
    grid_search.fit(X_train, y_train)

    print("Лучшие параметры: ", grid_search.best_params_)
    print("Лучший F1-score: ", grid_search.best_score_)

    return grid_search.best_estimator_

# Подбор гиперпараметров
best_model = hyperparameter_tuning(X_train, y_train)

# Оценка лучшей модели
y_pred_best = best_model.predict(X_test)
print(classification_report(y_test, y_pred_best))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Лучшие параметры:  {'clf__estimator__C': 10, 'tfidf__max_features': 5000}
Лучший F1-score:  0.9649673776264586
              precision    recall  f1-score   support

           0       0.99      0.97      0.98       327
           1       0.99      0.94      0.97      1635
           2       1.00      0.96      0.98       161
           3       1.00      0.99      1.00       139
           4       0.91      0.98      0.95       991

   micro avg       0.97      0.96      0.96      3253
   macro avg       0.98      0.97      0.97      3253
weighted avg       0.97      0.96      0.96      3253
 samples avg       0.96      0.96      0.96      3253



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [15]:
import numpy as np

def classify_query(query):
    query = preprocess_text(query)
    columns = ["Занятость", "По должности", "По дополнительному признаку", "По условиям", "Общая фраза"]
    predict_indexes = best_model.predict([query])[0]
    return predict_indexes

In [16]:
classify_query("вакансии мастер")
print(["Занятость", "По должности", "По дополнительному признаку", "По условиям", "Общая фраза"])
print(classify_query("вакансии мастер"))

['Занятость', 'По должности', 'По дополнительному признаку', 'По условиям', 'Общая фраза']
[0 0 0 0 1]
